##Connect to database! 
###Get Neighborhood Data

In [293]:
import pandas as pd
import json
import psycopg2
import random
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict

###Get tweets wtihin california, and shapes with the county of SF ('075')

In [294]:
conn_dict = {'dbname':'zipfiantwitter', 'user':'clwilloughby', 'password': '', 'host':'/tmp'}
conn = psycopg2.connect(dbname=conn_dict['dbname'], user=conn_dict['user'], host='/tmp')
cur = conn.cursor()

map_query = """
        SELECT
                DISTINCT
                ST_asGEOJSON(wkb_geometry) geometry
                , geoid10

                FROM sf_neighb 
                WHERE countyfp10 = '075'
                ;"""

geo = pd.read_sql(map_query, conn)

###format the geo pd

In [295]:
#intermediate step: not needed normally
#df.to_csv('sf_only_sql_shapes.csv')
#df = pd.read_csv('sql_shapes.csv')

In [296]:
geo['geoid10'] = geo.geoid10.astype('int64')
#set the index as the geoid
geo.set_index('geoid10', inplace=True)

###get tweet counts dataframe

In [299]:
#get the dataframe of counts from 'sql_tweets_map_retrieve'
df_tweet_count = pd.read_csv('test_count_df.csv')
df_tweet_count.drop('Unnamed: 0', 1, inplace=True)
#set the df as the index
df_tweet_count.set_index('geoid10', inplace=True)

###Merge SF Geometry with SF Tweets


In [301]:
map_df = pd.concat([geo, df_tweet_count], axis = 1)
#take out tweets that aren't in SF
#also if there are no tweets, fill with zero.
map_df = map_df[map_df.geometry.notnull()]
#fill empty counts with zero (no tweets)
map_df['count'].fillna(0, inplace = True)
map_df.reset_index(inplace=True)

###Get a Random Color For Each Neighborhood
####Later will encode information by color and hue

In [281]:
def getrandom_color(df, color_file):
    '''takes in a dataframe of gemoetry and geoid, and a list of hex colors
    Creates a new colomn of shape colors. Returns the new dataframe.'''
    #import the colors csv
    cdf = pd.read_csv('colors.csv')
    #make it a list
    colors = cdf[' color'].tolist() 
    num_col = df.shape[0] 
    df['hexcolor'] = [random.choice(colors) for i in xrange(num_col)]
    return df

In [222]:
df_col = getrandom_color(map_df, 'colors.csv')


/Users/christy/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [223]:
df_col.head()


,geoid10,geometry,count,text,hexcolors
0,60750101001000,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#CC0099
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",9,"[u'tfw', u'internet', u'archiv', u'sudden', u'...",#CCCCFF
2,60750101001002,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#3333CC
3,60750101001003,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#FF3399
4,60750101001004,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",6,"[u'deep', u'blue', u'adobelif', u'hearth', u'a...",#3333CC


###A. Convert DataFrame of Neighborhoods to JsonFile for Mapping

In [253]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolor']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

list_to_export = [add_properties_geo(row) for idx, row in df_col.iterrows()]

with open('sf_geo.json', 'w') as outfile:
    json.dump(list_to_export, outfile)

KeyError: 'hexcolor'

###B. Convert DataFrame of Conuts to Json File for Mapping

In [307]:
def add_properties_geo(row):
    geoid = row['geoid10']
    hexcolor = row['hexcolor']
    geo_json = {"type": "Feature", "geometry": json.loads(row['geometry']),  "properties": {'geoid': geoid , 'hexcolor' : hexcolor }}
    return geo_json

def dataframe_to_geojson(df, outfilename):
    '''Takes in a dataframe with a count and geoid10. Dumps it into a json geojason file'''
    df['geoid10'] = df['geoid10'].astype('str')
    
    list_to_export = [add_properties_geo(row) for idx, row in map_df.iterrows()]
    with open(outfilename, 'w') as outfile:
        json.dump(list_to_export, outfile)

####Manually heat map colors

In [120]:
color_list = ['#ff0000','#ff1919','#ff3232','#ff4c4c','#ff6666','#ff7f7f','#ff9999','#ffb2b2','#ffcccc','#ffe5e5']

In [348]:
df.head()

,geoid10,geometry,count,text,hexcolor
0,60750101001000,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#665229
1,60750101001001,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",9,"[u'tfw', u'internet', u'archiv', u'sudden', u'...",#806633
2,60750101001002,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#665229
3,60750101001003,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",0,NaN,#665229
4,60750101001004,"{""type"":""MultiPolygon"",""coordinates"":[[[[-122....",6,"[u'deep', u'blue', u'adobelif', u'hearth', u'a...",#806633


In [342]:
def fill_colors(df):
    '''Bulky way to create a manual heat map based on the counts of values.
    Need to find an automated way.
    Takes a dataframe with a COUNT column. Uses to return an added COLOR column
    which contains a spectrum of red colors.
    '''
    #create a dictionary of what color goes with each unique value

    values= float(len(df['count'].unique()))

    color_list = ['#665229','#806633','#997A3D','#B28F47','#CCA352','#E6B85C','#FFD175','#FFE0A3','#FFF0D1','#FFFAF0']
    value_dict = defaultdict(str)
    for index, numb in enumerate(df['count'].unique()):
        color= round(index/values*10)
        numb= int(numb)
        value_dict[numb] = color_list[int(color)]

    df['hexcolor'] = df['count'].map(value_dict.get)
    return df


In [343]:
df_color =fill_colors(map_df)
df_color = df_color[['geoid10', 'geometry', 'hexcolor']]
dataframe_to_geojson(df_color, 'data/heatmap_geo.json')

/Users/christy/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
